# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Kafka Producer: Financial Transaction Generator** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team members**: 
- Miguel Alberto Torres Dueñas
- Juan Pablo Cortez Navarro
- Luther Williams Sandria 
- Ferdinand Bierbaum

In [ ]:
import findspark
findspark.init()
#dbe4c53ffd27d3d2bdef46d2f12d349caadcfcb1438e112f

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://1c173c8c3406:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

In [11]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "d4f3f3b43802:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [12]:
from pyspark.sql.functions import split

kafka_df = kafka_lines.select(split(kafka_lines.value, " ").alias("traffic_array"))
kafka_df = kafka_df.withColumn("user_id", kafka_df["traffic_array"].getItem(0).cast("string"))
kafka_df = kafka_df.withColumn("video_id", kafka_df["traffic_array"].getItem(1).cast("string"))
kafka_df = kafka_df.withColumn("watch_time_seconds", kafka_df["traffic_array"].getItem(2).cast("integer"))
kafka_df = kafka_df.withColumn("resolution", kafka_df["traffic_array"].getItem(3).cast("string"))
kafka_df = kafka_df.withColumn("bitrate_kbps", kafka_df["traffic_array"].getItem(4).cast("integer"))
kafka_df = kafka_df.withColumn("buffering_events", kafka_df["traffic_array"].getItem(4).cast("integer"))
kafka_df = kafka_df.withColumn("paused", kafka_df["traffic_array"].getItem(4).cast("boolean"))
kafka_df = kafka_df.withColumn("skipped", kafka_df["traffic_array"].getItem(4).cast("boolean"))
kafka_df = kafka_df.withColumn("genre", kafka_df["traffic_array"].getItem(4).cast("string"))
kafka_df = kafka_df.withColumn("region", kafka_df["traffic_array"].getItem(4).cast("string"))
kafka_df = kafka_df.withColumn("recommended", kafka_df["traffic_array"].getItem(4).cast("boolean"))
kafka_df = kafka_df.withColumn("timestamp", kafka_df["traffic_array"].getItem(4).cast("timestamp"))


kafka_df.printSchema()

root
 |-- traffic_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- user_id: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- watch_time_seconds: integer (nullable = true)
 |-- resolution: string (nullable = true)
 |-- bitrate_kbps: integer (nullable = true)
 |-- buffering_events: integer (nullable = true)
 |-- paused: boolean (nullable = true)
 |-- skipped: boolean (nullable = true)
 |-- genre: string (nullable = true)
 |-- region: string (nullable = true)
 |-- recommended: boolean (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [13]:
query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(30)
query.stop()

25/05/10 20:15:09 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5592849d-e78f-45e9-847d-c0c770141976. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/10 20:15:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/10 20:15:09 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------+-------+--------+------------------+----------+------------+----------------+------+-------+-----+------+-----------+---------+
|traffic_array|user_id|video_id|watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre|region|recommended|timestamp|
+-------------+-------+--------+------------------+----------+------------+----------------+------+-------+-----+------+-----------+---------+
+-------------+-------+--------+------------------+----------+------------+----------------+------+-------+-----+------+-----------+---------+

